In [2]:
import numpy as np
import open3d as o3d

def main():
    pts = np.random.randint(0, 100, (100, 3))

    # whether to write in binary or text format
    write_text = True

    # use open3d
    use_o3d(pts, write_text)


def use_o3d(pts, write_text):
    pcd = o3d.geometry.PointCloud()

    # the method Vector3dVector() will convert numpy array of shape (n, 3) to Open3D format.
    # see http://www.open3d.org/docs/release/python_api/open3d.utility.Vector3dVector.html#open3d.utility.Vector3dVector
    pcd.points = o3d.utility.Vector3dVector(pts)

    # http://www.open3d.org/docs/release/python_api/open3d.io.write_point_cloud.html#open3d.io.write_point_cloud
    o3d.io.write_point_cloud("my_pts.ply", pcd, write_ascii=write_text)

    # read ply file
    pcd = o3d.io.read_point_cloud('my_pts.ply')

    # visualize
    o3d.visualization.draw_geometries([pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [26]:
main()

In [4]:
import copy
# read ply file
pcd = o3d.io.read_point_cloud('cropply/croppedtest2.ply')
pcd2 = o3d.io.read_point_cloud("cropply/croppedtest1.ply")
pcd_s = copy.deepcopy(pcd2).translate((2, 0, 0))

# visualize
o3d.visualization.draw_geometries([pcd,pcd_s])

In [87]:
from PIL import Image

# 이미지 열기
image_path = 'testpage/results/acc2/5test_Color.png'  # 이미지 파일 경로를 수정하세요
image = Image.open(image_path)

# 이미지 속성 확인
image_format = image.format
image_mode = image.mode
image_size = image.size

print(f"Format: {image_format}")
print(f"Mode: {image_mode}")
print(f"Size: {image_size}")

# 이미지 픽셀 값 확인 (RGB 이미지인 경우)
if image_mode == 'RGB':
    pixel_values = list(image.getdata())
    print(f"Pixel values of the first few pixels: {pixel_values[:10]}")
else:
    print("Pixel extraction is not supported for this image mode.")


Format: PNG
Mode: RGB
Size: (640, 480)
Pixel values of the first few pixels: [(38, 29, 34), (44, 33, 35), (54, 40, 40), (69, 59, 56), (82, 77, 71), (92, 94, 84), (98, 102, 92), (101, 105, 93), (101, 104, 89), (102, 104, 88)]


In [2]:
import numpy as np
import math

def yaw_pitch_roll_to_rotation_matrix(yaw, pitch, roll):
    # 각도를 라디안으로 변환
    yaw_rad = math.radians(yaw)
    pitch_rad = math.radians(pitch)
    roll_rad = math.radians(roll)

    # Yaw, Pitch, Roll 회전 행렬 계산
    yaw_matrix = np.array([
        [math.cos(yaw_rad), -math.sin(yaw_rad), 0],
        [math.sin(yaw_rad), math.cos(yaw_rad), 0],
        [0, 0, 1]
    ])

    pitch_matrix = np.array([
        [math.cos(pitch_rad), 0, math.sin(pitch_rad)],
        [0, 1, 0],
        [-math.sin(pitch_rad), 0, math.cos(pitch_rad)]
    ])

    roll_matrix = np.array([
        [1, 0, 0],
        [0, math.cos(roll_rad), -math.sin(roll_rad)],
        [0, math.sin(roll_rad), math.cos(roll_rad)]
    ])

    # 회전 행렬을 순서대로 곱해서 최종 회전 행렬 생성
    rotation_matrix = np.dot(np.dot(roll_matrix, pitch_matrix), yaw_matrix)
    return rotation_matrix

# 예시 각도 (단위: 도)
# yaw_angle = 30
# pitch_angle = 45
# roll_angle = 60
yaw_angle = 0
pitch_angle = 0
roll_angle = 90

# 회전 행렬 생성
rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
print(rotation_matrix)

[[ 1.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  6.123234e-17 -1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]]


In [34]:
import numpy as np
import open3d as o3d
import math

# read ply file
pcd = o3d.io.read_point_cloud('w640h360/2.ply')

# 
# rotate = np.identity(3)  
# size = np.ones(3) * 1.35 # trying to create a bounding box below 1 unit
# center = np.zeros(3) 

# 회전 행렬 참고자료 : https://en.wikipedia.org/wiki/Rotation_matrix
yaw_angle = 0  # + : crop 영역 반시계 방향 회전(카메라 시점), - : crop 영역 시계 방향 회전(카메라 시점)
pitch_angle = 0
roll_angle = 0

rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
size = [0.72,  # 너비(카메라 기준, 2d width) 1.139 보다 작아지면 우측 잘리기 시작함, 0.01 = 2 pixel 거리
        0.52,  # 높이(카메라 기준, 2d height) 0.849 보다 작아지면 아래쪽 잘리기 시작함, 0.00875 = 2 pixel 거리
        1.500]  # 깊이(카메라 기준 거리, 3d depth) 1.387 보다 작아지면 좌측 아래쪽 바닥 잘리기 시작함
                # 바닥 제거 1.345

center = [-0.025,  # + : crop 영역이 우측으로 이동, - : crop 영역이 좌측으로 이동 (-0.5195 ~ 0.5708), 합 1.0903
          0.0,  # + : crop 영역이 위쪽으로 이동, - : crop 영역이 아래쪽으로 이동 ( -0.4377 ~ 0.4179)
          0.0,] # + : crop 영역이 카메라쪽으로 이동, - : crop 영역이 아래쪽으로 이동
obb = o3d.geometry.OrientedBoundingBox(center, rotation_matrix, size) # or you can use axis aligned bounding box class

print(rotation_matrix)
print(size)
print(center)

crop_pcd = pcd.crop(obb)
print(pcd)
# visualize
o3d.visualization.draw_geometries([crop_pcd])
o3d.io.write_point_cloud("cropply/croppedtest2.ply", crop_pcd)


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0.72, 0.52, 1.5]
[-0.025, 0.0, 0.0]
PointCloud with 218935 points.


True

In [99]:
import numpy as np
import open3d as o3d

# 포인트 클라우드 파일 읽기
pcd = o3d.io.read_point_cloud('3dtest/5test_1.ply')

# 포인트 클라우드의 포인트 데이터 추출
points = np.asarray(pcd.points)

# 각 차원의 최소값과 최대값 구하기
min_x, min_y, min_z = np.min(points, axis=0)
max_x, max_y, max_z = np.max(points, axis=0)

# 넓이와 높이의 중심 좌표 구하기
width_center = (min_x + max_x) / 2
height_center = (min_y + max_y) / 2

print("Width Center:", width_center)
print("Height Center:", height_center)

Width Center: 0.09668000000000002
Height Center: -0.013427999999999995
